In [1]:
from xcube.core.store import new_data_store
import xarray as xr
import pandas as pd
import numpy as np
import os
import dask
import dask.array as da
from dask.distributed import Client, LocalCluster
from xcube_resampling import resample_in_time

In [2]:
cluster = LocalCluster(
    n_workers=8,
    threads_per_worker=2,
    memory_limit="3.5GB",   # per worker
)
client = Client(cluster)
client.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
%%time
team_store = new_data_store(
    "s3", 
    root=os.environ["S3_USER_STORAGE_BUCKET"], 
    storage_options=dict(
        anon=False, 
        key=os.environ["S3_USER_STORAGE_KEY"], 
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    )
)

CPU times: user 45.5 ms, sys: 49.1 ms, total: 94.5 ms
Wall time: 169 ms


In [4]:
team_store.list_data_ids()

['oc_cci.zarr',
 'oc_glo_cmems_300m.zarr',
 'oc_glo_cmems_4km.zarr',
 'oc_nws_cmems.zarr',
 'sen2_l2a.levels',
 'sen2_l2a_0.zarr',
 'sen2_l2a_1.zarr',
 'sen2_l2a_2.zarr',
 'sen2_l2a_3.zarr']

---

In [5]:
store = new_data_store("stac-pc-ardc")

In [6]:
store.list_data_ids()

['sentinel-2-l2a']

In [7]:
%%time
time_ranges = (
    ("2023-06-01", "2023-06-08"),
    ("2023-06-09", "2023-06-16"),
    ("2023-06-17", "2023-06-24"),
    ("2023-06-25", "2023-06-30"),
)

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.48 μs


In [8]:
%%time
for i, time_range in enumerate(time_ranges):
    if i < 3:
        continue
    ds = store.open_data(
        "sentinel-2-l2a",
        time_range=time_range,
        bbox=[4.3, 58.0, 6.5, 59.6],
        spatial_res=10 / 111320,
        crs="EPSG:4326",
        asset_names=["B02", "B03", "B04", "B08", "SCL"],
        tile_size=(2048, 2048),
    )
    team_store.write_data(ds,f"sen2_l2a_{i}.zarr", replace=True)
    print(f"Cube {i} written.")

/home/conda/users/f749afb9-1764838744-170-deepesdl-xcube-1.12.0/lib/python3.13/site-packages/xcube/core/store/fs/impl/dataset.py:200: UserWarning: Times can't be serialized faithfully to int64 with requested units 'seconds since 1970-01-01'. Resolution of 'milliseconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  data.to_zarr(
/home/conda/users/f749afb9-1764838744-170-deepesdl-xcube-1.12.0/lib/python3.13/site-packages/distributed/client.py:3374: UserWarning: Sending large graph of size 168.39 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/users/f749afb9-1764838744-170-deepesdl-xcube-1.12.0/lib/

Cube 3 written.
CPU times: user 8min 48s, sys: 36.3 s, total: 9min 25s
Wall time: 30min 51s


---

In [8]:
data_ids = [f"sen2_l2a_{i}.zarr" for i in range(4)]
dss = []
for data_id in data_ids:
    dss.append(team_store.open_data(data_id))

In [9]:
lat = dss[0]["lat"]
lon = dss[0]["lon"]
new_time = pd.to_datetime(["2023-06-01T10:00:00"])

lazy_nans = da.full(
    (1, dss[0].sizes["lat"], dss[0].sizes["lon"]),
    np.nan,
    chunks=dss[0].B02.data.chunksize
)
empty_slice = xr.DataArray(
    lazy_nans,
    dims=("time", "lat", "lon"),
    coords={"time": new_time, "lat": lat, "lon": lon},
)

# If your dataset has multiple variables, wrap in Dataset:
empty_ds = xr.Dataset(
    {var: empty_slice.copy() for var in ["B02", "B03", "B04", "B08"]}
)

lazy_zeros = da.full(
    (1, dss[0].sizes["lat"], dss[0].sizes["lon"]),
    0,
    chunks=dss[0].SCL.data.chunksize,
    dtype=np.uint8
)
empty_ds["SCL"] = xr.DataArray(
    lazy_zeros,
    dims=("time", "lat", "lon"),
    coords={"time": new_time, "lat": lat, "lon": lon},
)
empty_ds

<xarray.Dataset> Size: 14GB
Dimensions:  (time: 1, lat: 17813, lon: 24492)
Coordinates:
  * time     (time) datetime64[ns] 8B 2023-06-01T10:00:00
  * lat      (lat) float64 143kB 59.6 59.6 59.6 59.6 ... 58.0 58.0 58.0 58.0
  * lon      (lon) float64 196kB 4.3 4.3 4.3 4.3 4.3 4.3 ... 6.5 6.5 6.5 6.5 6.5
Data variables:
    B02      (time, lat, lon) float64 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03      (time, lat, lon) float64 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04      (time, lat, lon) float64 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08      (time, lat, lon) float64 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    SCL      (time, lat, lon) uint8 436MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [10]:
# 3) Concatenate the new slice in *front*
ds = xr.concat([empty_ds] + dss, dim="time")
ds

<xarray.Dataset> Size: 274GB
Dimensions:      (time: 19, lat: 17813, lon: 24492)
Coordinates:
  * time         (time) datetime64[ns] 152B 2023-06-01T10:00:00 ... 2023-06-3...
  * lat          (lat) float64 143kB 59.6 59.6 59.6 59.6 ... 58.0 58.0 58.0 58.0
  * lon          (lon) float64 196kB 4.3 4.3 4.3 4.3 4.3 ... 6.5 6.5 6.5 6.5 6.5
    spatial_ref  int64 8B 0
Data variables:
    B02          (time, lat, lon) float64 66GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, lat, lon) float64 66GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, lat, lon) float64 66GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08          (time, lat, lon) float64 66GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    SCL          (time, lat, lon) uint8 8GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [11]:
ds = ds.drop_vars("SCL")

In [12]:
%%time
agg_methods = {
    "B02": "mean",
    "B03": "mean",
    "B04": "mean",
    "B08": "mean",
    "SCL": "nearest",
}
ds = resample_in_time(ds, frequency="1D", agg_methods=agg_methods)
ds

CPU times: user 401 ms, sys: 23.8 ms, total: 425 ms
Wall time: 424 ms


<xarray.Dataset> Size: 419GB
Dimensions:      (lat: 17813, lon: 24492, time: 30)
Coordinates:
  * lat          (lat) float64 143kB 59.6 59.6 59.6 59.6 ... 58.0 58.0 58.0 58.0
  * lon          (lon) float64 196kB 4.3 4.3 4.3 4.3 4.3 ... 6.5 6.5 6.5 6.5 6.5
  * time         (time) datetime64[ns] 240B 2023-06-01 2023-06-02 ... 2023-06-30
    spatial_ref  int64 8B 0
Data variables:
    B02_mean     (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03_mean     (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04_mean     (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08_mean     (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    time_coverage_start:  <xarray.DataArray 'time' ()> Size: 8B\narray('2023-...
    time_coverage_end:    <xarray.DataArray 'time' ()> Size: 8B\narray('2023-...

In [13]:
%%time
ds = ds.rename(
    {
        "B02_mean": "B02",
        "B03_mean": "B03",
        "B04_mean": "B04",
        "B08_mean": "B08",
    }
)
ds

CPU times: user 288 μs, sys: 0 ns, total: 288 μs
Wall time: 292 μs


<xarray.Dataset> Size: 419GB
Dimensions:      (lat: 17813, lon: 24492, time: 30)
Coordinates:
  * lat          (lat) float64 143kB 59.6 59.6 59.6 59.6 ... 58.0 58.0 58.0 58.0
  * lon          (lon) float64 196kB 4.3 4.3 4.3 4.3 4.3 ... 6.5 6.5 6.5 6.5 6.5
  * time         (time) datetime64[ns] 240B 2023-06-01 2023-06-02 ... 2023-06-30
    spatial_ref  int64 8B 0
Data variables:
    B02          (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08          (time, lat, lon) float64 105GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    time_coverage_start:  <xarray.DataArray 'time' ()> Size: 8B\narray('2023-...
    time_coverage_end:    <xarray.DataArray 'time' ()> Size: 8B\narray('2023-...

In [14]:
%%time
team_store.write_data(ds, "sen2_l2a.levels", num_levels=5, replace=True)

CPU times: user 12min 39s, sys: 1min 45s, total: 14min 24s
Wall time: 55min 2s


'sen2_l2a.levels'